In [1]:
import pandas as pd
import numpy as np

sales_df = pd.read_excel('../dataSources/videoGames/Video game sales - 2000 - 2020.xlsx', sheet_name= 'Sheet1')

In [7]:
#Data splitting

from sklearn.model_selection import train_test_split
X = sales_df[['vg_score', 'critic_score', 'user_score']]
y = sales_df['global_sales']

# Define your feature matrix (X) and target variable (y)
# X should contain the independent variables, and y should contain the target variable (sales)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(f"Training set: {X_train.shape[0]} samples")
print(f"Testing set: {X_test.shape[0]} samples")

Training set: 7116 samples
Testing set: 1780 samples


In [4]:
#Linear Reg
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train) 

LinearRegression()

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Predict
y_pred = model.predict(X_test)

# metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R-squared (R2): {r2}')

MAE: 4.949108420692251
MSE: 33.16927999333766
RMSE: 5.759277731915493
R-squared (R2): 0.001878895347077414


In [6]:
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor()
model2.fit(X_train, y_train)

# Get feature importances
feature_importances = model2.feature_importances_